In [12]:
import requests
import pandas as pd
import psycopg2
from psycopg2 import sql    

uri = "https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date=20240819&selectType=ALL&response=json"
res = requests.get(uri)
data = res.json()['data']
data = [i + ["test"] for i in data]

In [14]:
import psycopg2
DB_HOST = 'localhost'
DB_NAME = 'tw'
DB_USER = 'mini'
DB_PASS = 'buddyrich134'
def get_db_connection():
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
    return conn

conn = get_db_connection()
cursor = conn.cursor()
cursor.executemany(f'''
INSERT INTO public.maincode (code, cname, ename, round_lot, outstanding_shares, equity_float, is_download_data, is_rt)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
''', data)
conn.commit()

In [17]:
cursor.execute("select cname from public.maincode where code = '1101';")
conn.commit()
cname = cursor.fetchone()[0]
cname

'台泥'